In [60]:
import pandas as pd
import numpy as np
import panel as pn
import hvplot.pandas
import calendar
import holoviews as hv
from bokeh.models.formatters import BasicTickFormatter

In [61]:
# Set some panel settings
#pn.extension('tabulator')
pn.config.sizing_mode = "stretch_width"
pn.widgets.Select(sizing_mode='stretch_width')

Select(sizing_mode='stretch_width')

In [62]:
#Color Data:
# These are the colors that will be used in the plot
color_sequence = ['#1f77b4', '#aec7e8', '#ff7f0e', '#ffbb78', '#2ca02c',
                  '#98df8a', '#d62728', '#ff9896', '#9467bd', '#c5b0d5',
                  '#8c564b', '#c49c94', '#e377c2', '#f7b6d2', '#7f7f7f',
                  '#c7c7c7', '#bcbd22', '#dbdb8d', '#17becf', '#9edae5']

In [63]:
# Graph Comparitive Data from all Regions
regions_list = [    
    "West Lake Marion",
    "Lebanon Hills", 
    "Murphy-Hanrehan", 
    "Battle Creek", 
    "Theodore Wirth", 
    "Xcel Energy",
]

region_name_map = {
  "battle-creek-5538": "Battle Creek",
  "west-lake-marion-park": "West Lake Marion",
  "lebanon-hills": "Lebanon Hills",
  "murphyhanrehan-park": "Murphy-Hanrehan",
  "theodore-wirth-park": "Theodore Wirth",
  "xcel-energy-mountain-bike-park-50624": "Xcel Energy"
}

all_region_df = pd.read_csv("data/region_ridecounts.csv")
all_region_df["estimated_riders"] = ((all_region_df["rides"] * .5) + all_region_df["rides"]).astype(int)
all_region_df["estimated_annual_revenue"] = ((all_region_df["estimated_riders"] * 5)*.25).astype(int)

for raw_region, clean_region in region_name_map.items():
    all_region_df["region"] = all_region_df["region"].replace({raw_region: clean_region})


In [64]:
multi_choice_1 = pn.widgets.MultiChoice(name='Region Selection', value=regions_list,
    options=regions_list)

In [65]:
# Load the dataframe into Cache
if 'data' not in pn.state.cache.keys():
    pn.state.cache['data'] = all_region_df.copy()

In [66]:
# Create an Interactive DataFrame
idf = all_region_df.interactive()

In [67]:
year_slider_1 = pn.widgets.IntSlider(name='Year slider', start=2016, end=2022, step=1, value=2016)
year_slider_1

df_g_y = (
    idf[
        (idf.year <= year_slider_1) &
        (idf.region.isin(multi_choice_1))
        ]
    .groupby(['year', 'region'])['estimated_riders'].sum()
    .to_frame()
    .reset_index()
    .sort_values(by='year')  
    .reset_index(drop=True)
)
df_g_y

region_plot_by_y = df_g_y.hvplot(
    kind='line', 
    x='year', 
    by='region', 
    title='Annual Count of Visitors by Region', 
    xlabel="Year", 
    ylabel="Visitors", 
    grid=True, 
    color=hv.Cycle(values=color_sequence),
    legend="left",
)
region_plot_by_y


C:\Users\faust\anaconda3\lib\site-packages\holoviews\core\data\pandas.py:221: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  data = [(k, group_type(v, **group_kwargs)) for k, v in
C:\Users\faust\anaconda3\lib\site-packages\holoviews\core\data\pandas.py:221: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  data = [(k, group_type(v, **group_kwargs)) for k, v in


In [68]:
year_slider_2 = pn.widgets.IntSlider(name='Year-Month slider', start=2015, end=2022, step=1, value=2015)
year_slider_2
multi_choice_2 = pn.widgets.MultiChoice(name='Region Selection', value=regions_list,
    options=regions_list)

df_g_ym = (
    idf[
        (idf.year == year_slider_2) &
        (idf.region.isin(multi_choice_2))
        ]
    .groupby(['month', 'region'])['estimated_riders'].sum()
    .to_frame()
    .reset_index()
    .sort_values(by='month')  
    .reset_index(drop=True)
)

region_plot_by_ym = df_g_ym.hvplot(
    kind='line', 
    x='month', 
    by='region', 
    title='Count of Visitors by Region', 
    xlabel="Month", 
    ylabel="Visitors", 
    grid=True, 
    color=hv.Cycle(values=color_sequence),
    legend="left",
)
region_plot_by_ym

C:\Users\faust\anaconda3\lib\site-packages\holoviews\core\data\pandas.py:221: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  data = [(k, group_type(v, **group_kwargs)) for k, v in
C:\Users\faust\anaconda3\lib\site-packages\holoviews\core\data\pandas.py:221: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  data = [(k, group_type(v, **group_kwargs)) for k, v in


In [69]:
# Let's play with potential Revenue and dollars into Lakeville
## For hypothetical purposes, let's say 1/4 of the riders spend on average $5/visit to WLMT on local businesses
dollars_g = (
    idf
    .groupby(['year', 'region'])['estimated_annual_revenue'].sum()
    .to_frame()
    .reset_index()
    .sort_values(by='year')  
    .reset_index(drop=True)
)

annual_revenue = dollars_g.hvplot(
    kind='bar', 
    x='year', 
    by='region', 
    stacked=True, 
    grid=True,  
    title="Estimated Revenue by Year", 
    color=hv.Cycle(values=color_sequence), 
    xlabel="Year", 
    ylabel="Estimated Annual Revenue", 
    yformatter='$%.2f',
    legend="left",
)
annual_revenue

In [70]:
tmp_df_0 = all_region_df[all_region_df['region']== "West Lake Marion"]
plain_df = (tmp_df_0.groupby(['year', 'region'])['estimated_riders'].sum() / 4).reset_index()
plain_df["$5_per_rider"] = (plain_df["estimated_riders"] * 5).astype(int)
plain_df["$10_per_rider"] = (plain_df["estimated_riders"] * 10).astype(int)
plain_df["$15_per_rider"] = (plain_df["estimated_riders"] * 15).astype(int)
plain_df["$20_per_rider"] = (plain_df["estimated_riders"] * 20).astype(int)
plain_df.reset_index()
plain_df = plain_df.drop('estimated_riders', axis=1)
plain_df = plain_df[plain_df['year'] >= 2017]

plain_idf = (
    plain_df
    .sort_values(by='year')
).interactive()
plain_idf

dollars_marion = plain_idf.hvplot(
    kind='line', 
    x='year', 
    grid=True, 
    ylabel="Estimated Annual Revenue", 
    xlabel="Year",
    yformatter='$%.2f',
    title="Estimate of Current Revenue by Rider Count & Amount Spent -- ((sum_of_riders_year * .25) * $_spent_estimate)",
)

dollars_marion

In [71]:
# Narrow down to Marion into an interactive DF
tmp_df_1 = all_region_df[all_region_df['region']== "West Lake Marion"]
wlmt_idf = tmp_df_1.interactive()

In [72]:
# Populatiry of West Lake Marion by Year
df_g_1 = wlmt_idf.groupby(tmp_df_1['year'], as_index=False)['estimated_riders'].sum()

marion_populatiry_by_year = df_g_1.hvplot(
    kind='bar', 
    x='year', 
    title='Visitors to WLMT by Year', 
    ylabel='Number of Visitors', 
    xlabel="Year", 
    grid=True, 
    color=hv.Cycle(values=color_sequence)
)
marion_populatiry_by_year

In [73]:
# Populatiry of West Lake Marion by Month
df_g_2 = wlmt_idf.groupby(['month','month_name'])['estimated_riders'].sum()
marion_populatiry_by_month = df_g_2.hvplot(
    kind='bar', 
    x='month_name', 
    title='Visitors to WLMT by Month', 
    ylabel='Number of Visitors', 
    xlabel="Month", 
    grid=True, 
    color=hv.Cycle(values=color_sequence)
)
marion_populatiry_by_month

In [74]:
# start simply by just brining in the user/rider data to play with
riders_df = pd.read_csv("./data/wlmt_riders.csv")

# Narrow down to riders who've ridding WLMT
region = "West Lake Marion"
marion_mask = riders_df.recent_ride_locations.apply(lambda x: region in x)
region_df = riders_df[marion_mask]
region_df = region_df[region_df['state'] != "unknown"]
region_df = region_df[region_df['city'] != "unknown"]

# Load the dataframe into cache
pn.state.cache['data'] = region_df.copy()

In [75]:
region_idf = region_df.interactive()

In [76]:
# Graph Riders of WLMT by all states in the USA
# Measure visitors of trails by all states

visit_by_state = (
    ((region_idf.groupby('state').size().sort_values(ascending=False) / region_idf.groupby('state').size().sort_values(ascending=False).sum()) * 100)
    .to_frame()
    .reset_index()
)

visitors_plot_by_state = visit_by_state.hvplot(
    kind='bar', 
    x='state',
    title='% of WLMT Visitors by State', 
    ylabel='% of Riders', 
    xlabel="State", 
    grid=True, 
    color=hv.Cycle(values=color_sequence)
)
visitors_plot_by_state

In [77]:
# Graph Riders of WLMT By City
#g_2 = (region_df.groupby('city').size().sort_values(ascending=False) / region_df.groupby('city').size().sort_values(ascending=False).sum())*100

visit_by_city = (
    ((region_idf.groupby('city').size().sort_values(ascending=False) / region_idf.groupby('city').size().sort_values(ascending=False).sum()) * 100)
    .to_frame()
    .reset_index()
)


visitors_plot_by_city = visit_by_city.hvplot(
    kind='bar', 
    x='city', 
    title='% of WLMT Visitors by City', 
    ylabel='% of Riders', 
    xlabel="City",
    rot=90, 
    grid=True, 
    color=hv.Cycle(values=color_sequence),
    
)

visitors_plot_by_city

In [78]:
# Let's play with the Region Trail Data
# Build a Pandas Data Frame with all region data

region_trails_df = pd.read_csv("data/region_trails.csv")
region_trails_df['region_title'] = region_trails_df['region_title'].replace({"Xcel Energy Mountain Bike Park": "Xcel Energy", "West Lake Marion Park":"West Lake Marion"})
region_trails_df['region_name'] = region_trails_df['region_title']
region_trails_df['trail_type_count'] = region_trails_df['difficulty']

region_trails_df.to_csv("test.csv")

In [79]:
trail_idf = region_trails_df.interactive()

In [80]:
trail_types = ["Green", "Blue", "Black Diamond", "Double Black Diamond"]

region_difficulty_df = (
    trail_idf[
        (trail_idf.difficulty.isin(trail_types))
    ]
    .groupby(['region_title', 'difficulty'])['trail_type_count'].count()
    .to_frame()
    .reset_index()
    .sort_values(by='region_title')  
    .reset_index(drop=True)
)
trail_type_by_region = region_difficulty_df.hvplot(
    kind='bar', 
    x='region_title', 
    by='difficulty', 
    xlabel="Region", 
    height=410, 
    grid=True, 
    ylabel="Total Number of Trails", 
    title="Total Trails by Region and Difficulty", 
    stacked=True, 
    cmap=['#454844', 'blue', "green", "black"]
)
trail_type_by_region

In [81]:
#region_miles_count = region_trails_df.groupby('region_title')['distance_miles'].sum().sort_values()
region_miles_all_count = (
    region_trails_df.groupby('region_title')['total_miles'].sum().sort_values(ascending=False).reset_index()
)

region_miles_descent_count = (
    region_trails_df.groupby('region_title')['descent_miles'].sum().sort_values(ascending=False).reset_index()
)

region_miles_ascent_count = (
    region_trails_df.groupby('region_title')['ascent_miles'].sum().sort_values(ascending=False).reset_index()
)

region_miles_flat_count = (
    region_trails_df.groupby('region_title')['flat_miles'].sum().sort_values(ascending=False).reset_index()
)

region_miles_count_concat = pd.concat([region_miles_all_count, region_miles_descent_count, region_miles_ascent_count, region_miles_flat_count])
region_miles_count_concat.set_index('region_title')
region_miles_count = region_miles_count_concat.groupby('region_title', as_index=False).sum().interactive()
miles_of_trail_by_region = region_miles_count.hvplot(
    kind='bar', 
    x='region_title', 
    rot=40, 
    legend="top_left", 
    grid=True, 
    title="Miles of Trail Available by Region and Ride Slope",
    color=hv.Cycle(values=color_sequence),
    xlabel="Region  and Miles by (Descent, Ascent, Flat)",
    ylabel="Miles"
)
miles_of_trail_by_region

C:\Users\faust\anaconda3\lib\site-packages\holoviews\core\data\pandas.py:221: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  data = [(k, group_type(v, **group_kwargs)) for k, v in
C:\Users\faust\anaconda3\lib\site-packages\holoviews\core\data\pandas.py:221: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  data = [(k, group_type(v, **group_kwargs)) for k, v in


In [82]:
#Custom CSS:
#.bk-root .choices__list--dropdown .choices__item--selectable

css = '''
.choices__item .choices__item--selectable {
    background-color: darkslategray;
    text-color: #149414;
}
.choices__item:hover .choices__item--selectable:hover {
    background-color: black;
    text-color: white;
}

.bk-root .choices__list--dropdown .choices__item--selectable.is-highlighted {
    background-color: black;
    text-color: green;
}
'''

link_color_css = """
   a:link {color: darkorange;}  
   a:visited {color: darkorange;}  
   a:hover {color: #FCFC0C;}  
"""


pn.extension(raw_css=[css, link_color_css])


# Markdown Sections
links_md = """
- [MN-MTB](http://mn-mtb.com)
- [MN-MTB Trailforks](https://www.trailforks.com/profile/mnmtb/)
- [Lakeville Cycling Assocation](https://www.lakevillecycling.org/)
- [Lakeville MTB Team](https://lakevillemountainbiketeam.shutterfly.com/)

Contact us: [MinnMTB@gmail.com](mailto:MinnMTB@gmail.com)
"""

terms_md = """
#### Terms

- **MTB** - Mountain Biking
- **Region** - A region as defined by Trailforks (e.g, West Lake Marion Trails, Buck Hill, Battle Creek, etc.)
- **Revenue Estimations** - For each visit/ride for a region, 1 of 4 riders will spend $5 in the local city/municipality.
    - `(region_rides_sum / 4) * 5`
- **WLMT** - West Lake Marion Trails
"""

data_updated_md = """
All data seen on this dashboard was obtained via public or administrative means on [Trailforks.com](https://www.trailforks.com).

Data Last Updated: 10.14.2022
"""

def reset_widgets(key):
    multi_choice_1.value = regions_list
    multi_choice_2.value = regions_list
    multi_choice_3.value = regions_list
    return 0

reset_widgets_button = pn.widgets.Button(name="Reset Region Selection")
reset_widgets_button.on_click(reset_widgets)
reset_widgets_button


Button(name='Reset Region Selection', sizing_mode='stretch_width')

In [83]:
#Layout using Template
# https://panel.holoviz.org/api/panel.pane.html?highlight=pane%20markdown#pane-package
#https://panel.holoviz.org/reference/panes/PNG.html

template = pn.template.MaterialTemplate(
    theme = pn.template.theme.DarkTheme,
    title='MN MTB Region Metrics',

    sidebar=[
        pn.pane.Markdown("# MN MTB Metrics"), 
        pn.pane.PNG('images/mnmtb.png', width=300),
        pn.pane.Markdown(links_md),
        pn.layout.Divider(),
        pn.pane.Str("Controls", style={'font-size': '1vw', "margin": "5px", "width": "100%", "text-align": "center", "font-family": "Verdana, Arial, Tahoma, Serif", "text-decoration": " solid white"}),
        reset_widgets_button,
        pn.layout.Divider(),
        pn.pane.Markdown(terms_md),
        pn.layout.Divider(),
        pn.pane.Markdown(data_updated_md),
    ],
    main=[

        pn.pane.Str('Region Comparisons', style={'font-size': '1.5vw', "margin": "auto",
  "width": "100%", "text-align": "center", "font-family": "Verdana, Arial, Tahoma, Serif", "text-decoration": "solid white", "color":"darkorange"}),
        
        # Row 1
        pn.Row(
            pn.Column(region_plot_by_y, sizing_mode="stretch_width", width_policy='max'), 
            pn.Column(region_plot_by_ym, sizing_mode="stretch_width", width_policy='max'),
            css_classes=['choices', 'choices__item', 'choices__list', 'choices:hover'],
            sizing_mode="stretch_width",
            width_policy='max',
        ), 
        
        pn.Row(
            pn.Column(annual_revenue),
        ), 
        
        pn.Row(
            pn.Column(miles_of_trail_by_region),
        ), 
               
        pn.Row(
            pn.Column(trail_type_by_region),
        ),
        
        pn.layout.Divider(),
        pn.pane.Str('West Lake Marion Visitors', style={'font-size': '1.5vw', "margin": "auto",
  "width": "100%", "text-align": "center", "font-family": "Verdana, Arial, Tahoma, Serif", "text-decoration": "solid white", "color":"darkorange"}),
        
        # Row 2
        pn.Row(
            pn.Column(visitors_plot_by_state), 
            pn.Column(visitors_plot_by_city),

        ),
        
        
        pn.layout.Divider(),
        pn.pane.Str('West Lake Marion Popularity', style={'font-size': '1.5vw', "margin": "auto",
  "width": "100%", "text-align": "center", "font-family": "Verdana, Arial, Tahoma, Serif", "text-decoration": "solid white", "color":"darkorange"}),
        
        # Row 3
        pn.Row(
            pn.Column(marion_populatiry_by_year), 
            pn.Column(marion_populatiry_by_month),

        ),
        
        pn.layout.Divider(),
        pn.pane.Str('West Lake Marion Revenue', style={'font-size': '1.5vw', "margin": "auto",
  "width": "100%", "text-align": "center", "font-family": "Verdana, Arial, Tahoma, Serif", "text-decoration": "solid white", "color":"darkorange"}),
        
        pn.Row(
            pn.Column(dollars_marion),
        ),
        
    ],
)
template.servable();